`Learning Advanced API Concepts`

- [Learning Advanced API Concepts](https://realpython.com/python-api/)

There are a few more advanced topics that are worth touching upon, even if briefly, such as `authentication`, `pagination`, and `rate` limiting.

`Authentication`

- `API Key`: The most common level of authentication is the API key.
- These keys are used to identify you as an API user or customer and to trace your use of the API. API keys are typically sent as a request header or as a query parameter.

[NASA APIs](https://api.nasa.gov/)

In [11]:
import os
import configparser

# Read credentials.cfg file
config = configparser.ConfigParser()
config.read('credentials.cfg')

NASA_API_KEY = config['API']['NASA']
OPENWEATHER_API_KEY = config['API']['OPENWEATHER']
GITHUB_CLIENT_ID = config['API']['GITHUB_CLIENTID']
GITHUB_CLIENT_SECRETS = config['API']['GITHUB_CLIENTSECRETS']

In [4]:
import requests

endpoint = "https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos"
api_key = NASA_API_KEY
query_params = {"api_key": api_key, "earth_date":"2020-07-01"} # You can add the API key to your request by appending the api_key= query parameter:

response = requests.get(endpoint, params=query_params)
response.status_code

200

In [7]:
# Now have a look at the Response object and try to extract some pictures from it:
response.json()

{'photos': [{'id': 754118,
   'sol': 2809,
   'camera': {'id': 20,
    'name': 'FHAZ',
    'rover_id': 5,
    'full_name': 'Front Hazard Avoidance Camera'},
   'img_src': 'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/02809/opgs/edr/fcam/FLB_646868981EDR_F0810628FHAZ00337M_.JPG',
   'earth_date': '2020-07-01',
   'rover': {'id': 5,
    'name': 'Curiosity',
    'landing_date': '2012-08-06',
    'launch_date': '2011-11-26',
    'status': 'active',
    'max_sol': 4060,
    'max_date': '2024-01-07',
    'total_photos': 692166,
    'cameras': [{'name': 'FHAZ', 'full_name': 'Front Hazard Avoidance Camera'},
     {'name': 'NAVCAM', 'full_name': 'Navigation Camera'},
     {'name': 'MAST', 'full_name': 'Mast Camera'},
     {'name': 'CHEMCAM', 'full_name': 'Chemistry and Camera Complex'},
     {'name': 'MAHLI', 'full_name': 'Mars Hand Lens Imager'},
     {'name': 'MARDI', 'full_name': 'Mars Descent Imager'},
     {'name': 'RHAZ', 'full_name': 'Rear Hazard Avoidance Camera'

In [8]:
response.json()['photos']

[{'id': 754118,
  'sol': 2809,
  'camera': {'id': 20,
   'name': 'FHAZ',
   'rover_id': 5,
   'full_name': 'Front Hazard Avoidance Camera'},
  'img_src': 'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/02809/opgs/edr/fcam/FLB_646868981EDR_F0810628FHAZ00337M_.JPG',
  'earth_date': '2020-07-01',
  'rover': {'id': 5,
   'name': 'Curiosity',
   'landing_date': '2012-08-06',
   'launch_date': '2011-11-26',
   'status': 'active',
   'max_sol': 4060,
   'max_date': '2024-01-07',
   'total_photos': 692166,
   'cameras': [{'name': 'FHAZ', 'full_name': 'Front Hazard Avoidance Camera'},
    {'name': 'NAVCAM', 'full_name': 'Navigation Camera'},
    {'name': 'MAST', 'full_name': 'Mast Camera'},
    {'name': 'CHEMCAM', 'full_name': 'Chemistry and Camera Complex'},
    {'name': 'MAHLI', 'full_name': 'Mars Hand Lens Imager'},
    {'name': 'MARDI', 'full_name': 'Mars Descent Imager'},
    {'name': 'RHAZ', 'full_name': 'Rear Hazard Avoidance Camera'}]}},
 {'id': 754119,
  'sol': 28

In [9]:
photos = response.json()['photos']
print(f"Found {len(photos)} photos")

Found 12 photos


In [10]:
# For this example, you picked a specific earth_date (2020-07-01) and then a specific photo from the response dictionary (4). 
photos[4]["img_src"]

'https://mars.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/02809/opgs/edr/rcam/RRB_646869036EDR_F0810628RHAZ00337M_.JPG'

`OAuth: Getting Started`

`Example using the GitHub API!`

- [GitHub Creating an OAuth app](https://docs.github.com/en/apps/oauth-apps/building-oauth-apps/creating-an-oauth-app)

- [GitHub API Doc](https://docs.github.com/en/rest?apiVersion=2022-11-28)

In [ ]:
#GITHUB_CLIENT_ID = config['API']['GITHUB_CLIENTID']
#GITHUB_CLIENT_SECRETS = config['API']['GITHUB_CLIENTSECRETS']

Once you’ve created your app, copy and paste the Client_ID and Client_Secret, together with your selected redirect URL, into a Python file called github.py:

In [12]:
import requests

# REPLACE the following variables with your Client ID and Client Secret
CLIENT_ID = GITHUB_CLIENT_ID
CLIENT_SECRET = GITHUB_CLIENT_SECRETS

# REPLACE the following variable with what you added in the
# "Authorization callback URL" field
REDIRECT_URI = "https://httpbin.org/anything"

you need to be able to create a link to redirect the user to their GitHub account, as the GitHub documentation explains:

- [Request a user's GitHub identity](https://docs.github.com/en/apps/oauth-apps/building-oauth-apps/authorizing-oauth-apps#1-request-a-users-github-identity)

In [13]:
# In this piece of code, you first define the required parameters that the API expects and then call the API using the requests package and .get().
def create_oauth_link():
    params = {
        "client_id":CLIENT_ID
       ,"redirect_uri":REDIRECT_URI
       ,"scope":"user"
       ,"response_type":"code"
    }

    endpoint = "https://github.com/login/oauth/authorize"
    response = requests.get(endpoint, params=params)
    return response.url

When you make the request to the /login/oauth/authorize endpoint, the API will automatically redirect you to the GitHub website. 
In that case, you want to fetch the url parameter from the response. This parameter contains the exact URL that GitHub is redirecting you to.

The next step in the authorization flow is to exchange the code you get for an access token. Again, following the steps in GitHub’s documentation, you can make a function for it:

- [Users are redirected back to your site by GitHub](https://docs.github.com/en/apps/oauth-apps/building-oauth-apps/authorizing-oauth-apps#2-users-are-redirected-back-to-your-site-by-github)

In [15]:
# Here, you make a POST request to exchange the code for an access token. In this request, you have to send your CLIENT_SECRET and code so that GitHub can validate that this specific code was initially generated by your application. 
# Only then will the GitHub API generate a valid access token and return it to you.
def exchange_code_for_access_token(code=None):
    params = {
        "client_id":CLIENT_ID
       ,"client_secret":CLIENT_SECRET
       ,"redirect_uri":REDIRECT_URI
       ,"code":code
    }

    headers = {"Accept":"application/json"}
    endpoint = "https://github.com/login/oauth/access_token"
    response = requests.post(endpoint, params=params, headers=headers).json()
    return response["access_token"]

In [25]:
# If everything goes according to plan, then you should be rewarded with a valid access token that you can use to make calls to the GitHub API, impersonating the authenticated user.
link = create_oauth_link()
#print(f"Follow the link to start the authentication with GitHub: {link}")
code = input("GitHub code: ")
access_token = exchange_code_for_access_token(code)
#print(f"Exchanged code {code} with access token: {access_token}")

KeyError: 'access_token'

Now try adding the following code to fetch your user profile using the User API and to print your name, username, and number of private repositories:
- [User API](https://docs.github.com/en/rest/users?apiVersion=2022-11-28#get-the-authenticated-user)

In [24]:
# Update the access_token variable with the access token that you received from the previous step:
access_token = ''

In [27]:

headers = {"Authorization":f"token {access_token}"}
endpoint = "https://api.github.com/user"
response = requests.get(endpoint, headers=headers).json()
#response

In [22]:
def print_user_info(access_token=None):
    headers = {"Authorization":f"token {access_token}"}
    endpoint = "https://api.github.com/user"
    response = requests.get(endpoint, headers=headers).json()
    name = response["name"]
    username = response["login"]
    private_repos_count = response["total_private_repos"]
    print(f"{name} ({username}) | private repositories: {private_repos_count}")

In [29]:
#print_user_info(access_token)

`Pagination`

In very simple terms, `pagination` is the act of splitting large amounts of data into multiple smaller pieces. For example, whenever you go to the questions page in Stack Overflow, you see something like this at the bottom:

`For APIs in particular, this is normally handled with the help of query parameters, mainly the following two:`

 - A `page` attribute that defines which page you’re currently requesting
 - A `size` attribute that defines the size of each page


- [GitHub Public Events API](https://docs.github.com/en/rest/activity?apiVersion=2022-11-28#list-public-events)

In [32]:
# Using the GitHub API again, you can find an events endpoint in the documentation that contains pagination query parameters. 
# The parameter per_page= defines the number of items to return, and page= allows you to paginate through multiple results. Here’s how to use these parameters:

response = requests.get("https://api.github.com/events?per_page=1&page=0")
response.json()[0]['id']

'34689232177'

In [33]:
response = requests.get("https://api.github.com/events?per_page=1&page=1")
response.json()[0]['id']

'34689243505'

In [35]:
response = requests.get("https://api.github.com/events?per_page=1&page=2")
response.json()[0]['id']

'34689263460'

`Rate Limiting`

`For the sake of demonstration, you’ll purposefully try to exceed GitHub’s rate limit to see what happens. In the code below, you’ll request data until you get a status code other than 200 OK:`

In [36]:
endpoint = "https://api.github.com/events"
for i in range(1,6):
    response = requests.get(endpoint, params={"per_page":1,"page":i})
    print(f"{i}: {response.status_code}")
    if response.status_code != 200:
        break

1: 200
2: 200
3: 200
4: 200
5: 200


In [37]:
# Have a look at the latest response.headers to see if you can find those specific rate-limiting headers.
response.headers

{'Server': 'GitHub.com', 'Date': 'Tue, 09 Jan 2024 17:39:48 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': 'W/"5587b40be633d36bc140fff137f6dae942d5bb8155ea100d9bf303b6ecd165a3"', 'Last-Modified': 'Tue, 09 Jan 2024 17:34:48 GMT', 'X-Poll-Interval': '60', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Link': '<https://api.github.com/events?per_page=1&page=4>; rel="prev", <https://api.github.com/events?per_page=1&page=6>; rel="next", <https://api.github.com/events?per_page=1&page=274>; rel="last", <https://api.github.com/events?per_page=1&page=1>; rel="first"', 'x-github-api-version-selected': '2022-11-28', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Med

`Consuming APIs With Python: Practical Examples`

Searching and Fetching Trending GIFs
How about making a small script that fetches the top three trending GIFs from the GIPHY website? To do this, you need to create an app and get an API key from GIPHY. You can find instructions by expanding the box below, and you can also check GIPHY’s quickstart documentation.

- [giphy](https://giphy.com/)
- [giphy API Doc](https://developers.giphy.com/docs/api/#quick-start-guide)
- [giphy API Explorer](https://developers.giphy.com/explorer/)

In [38]:
import os
import configparser

# Read credentials.cfg file
config = configparser.ConfigParser()
config.read('credentials.cfg')

NASA_API_KEY = config['API']['NASA']
OPENWEATHER_API_KEY = config['API']['OPENWEATHER']
GITHUB_CLIENT_ID = config['API']['GITHUB_CLIENTID']
GITHUB_CLIENT_SECRETS = config['API']['GITHUB_CLIENTSECRETS']
GIPHY_API_KEY = config['API']['GIPHY']

In [39]:
import requests

# Replace the following with the API Key generated from the Giphy API
# you define the params and add your own API key. You also include a couple of other filters: limit to get 3 results and rating to get only appropriate content.
API_KEY = GIPHY_API_KEY
endpoint = "https://api.giphy.com/v1/gifs/trending"

params = {"api_key":API_KEY, "limit":3, "rating":"g"}
response = requests.get(endpoint, params=params)
response.status_code 


200

In [44]:
response.headers.get('Content-Type')

'application/json'

In [45]:
response.json()

{'data': [{'type': 'gif',
   'id': 'gRhW4YgNvO6dLeLTf8',
   'url': 'https://giphy.com/gifs/primevideo-prime-video-saltburn-salt-burn-gRhW4YgNvO6dLeLTf8',
   'slug': 'primevideo-prime-video-saltburn-salt-burn-gRhW4YgNvO6dLeLTf8',
   'bitly_gif_url': 'https://gph.is/g/4D0DXMB',
   'bitly_url': 'https://gph.is/g/4D0DXMB',
   'embed_url': 'https://giphy.com/embed/gRhW4YgNvO6dLeLTf8',
   'username': 'Saltburnfilm',
   'source': '',
   'title': 'Amazon Thank You GIF by Saltburn',
   'rating': 'g',
   'content_url': '',
   'source_tld': '',
   'source_post_url': '',
   'is_sticker': 0,
   'import_datetime': '2024-01-05 18:58:47',
   'trending_datetime': '2024-01-09 18:00:07',
   'images': {'original': {'height': '360',
     'width': '480',
     'size': '723813',
     'url': 'https://media4.giphy.com/media/gRhW4YgNvO6dLeLTf8/giphy.gif?cid=a49abec2b8tg78kmu8g4pv7n57ulcvqsqsljztmnfdwodahl&ep=v1_gifs_trending&rid=giphy.gif&ct=g',
     'mp4_size': '88655',
     'mp4': 'https://media4.giphy.com/med

In [46]:
for gif in response.json()["data"]:
    title = gif["title"]
    trending_date = gif["trending_datetime"]
    url = gif["url"]
    print(f"{title} | {trending_date}\n{url}\n")

Amazon Thank You GIF by Saltburn | 2024-01-09 18:00:07
https://giphy.com/gifs/primevideo-prime-video-saltburn-salt-burn-gRhW4YgNvO6dLeLTf8

Happy Birthday Reaction GIF by The Office | 2021-07-06 21:46:51
https://giphy.com/gifs/theoffice-happy-birthday-the-office-happybirthday-g5R9dok94mrIvplmZd

Be Kind To Yourself Mental Health GIF by INTO ACTION | 2024-01-09 17:55:10
https://giphy.com/gifs/IntoAction-mental-health-love-yourself-take-care-of-qbYsmyCoHPdfcs4dm1



In [1]:
import os
import configparser
import requests

# Read credentials.cfg file
config = configparser.ConfigParser()
config.read('credentials.cfg')
GIPHY_API_KEY = config['API']['GIPHY']


# REPLACE the following variable with your API Key
API_KEY = GIPHY_API_KEY
#Emoji: https://api.giphy.com/v2/emoji?api_key=&limit=25&offset=0
endpoint = "https://api.giphy.com/v2/emoji"
params = {"api_key": API_KEY, "limit": 3, "offset": 0}
response = requests.get(endpoint, params=params).json()

In [2]:
for gif in response["data"]:
    title = gif["title"]
    trending_date = gif["trending_datetime"]
    url = gif["url"]
    print(f"{title} | {trending_date}\n{url}\n")

You Got It Yes Sticker by Emoji | 2022-09-07 17:15:10
https://giphy.com/stickers/Emoji-thumbs-up-emoji-animated-dalJ0CpF7hwmN1nZXe

Wash Hands Corona Sticker by Emoji | 2020-09-17 20:00:03
https://giphy.com/stickers/Emoji-emoji-wash-hand-washing-ftklTmdD9MN3uXaLvb

Peace Hands Sticker by Emoji | 2021-04-01 05:00:05
https://giphy.com/stickers/Emoji-transparent-4tSHBpzJw7R3rrKUeo



```Text
Now, say you want to make a script that allows you to search for a specific word and fetch the first GIPHY match for that word. A different endpoint and slight variation of the code above can do that quite quickly:
```

In [3]:
import os
import configparser
import requests

# Read credentials.cfg file
config = configparser.ConfigParser()
config.read('credentials.cfg')
GIPHY_API_KEY = config['API']['GIPHY']

# REPLACE the following variable with your API Key
API_KEY = GIPHY_API_KEY
#Search: https://api.giphy.com/v1/gifs/search?api_key=r&q=&limit=25&offset=0&rating=g&lang=en&bundle=messaging_non_clips
endpoint = "https://api.giphy.com/v1/gifs/search"

search_term = "shrug"
params = {"api_key": API_KEY, "limit": 1, "q": search_term, "rating": "g"}
response = requests.get(endpoint, params=params).json()
for gif in response["data"]:
    title = gif["title"]
    url = gif["url"]
    print(f"{title} | {url}")

Season 5 Idk GIF by Paramount+ | https://giphy.com/gifs/CBSAllAccess-season-5-episode-23-i-love-lucy-JRhS6WoswF8FxE0g2R


`Searching Google Books`

- For this example, you’ll use the `Google Books API` and the `public volumes endpoint` to search for books.
  - [Google Books API](https://developers.google.com/books)
  - [Public Volumes Endpoint](https://developers.google.com/books/docs/v1/using#WorkingVolumes)

In [ ]:
# https://www.googleapis.com/books/v1/volumes?q=search+terms
# https://www.googleapis.com/books/v1/volumes?q=flowers+inauthor:keyes&key=yourAPIKey

In [3]:
# Here’s a straightforward piece of code to look for the words "moby dick" in the whole catalog:
import os
import configparser
import requests

# Read credentials.cfg file
config = configparser.ConfigParser()
config.read('credentials.cfg')
GBOOKS_API_KEY = config['API']['GOOGLE_BOOKS']

# REPLACE the following variable with your API Key
API_KEY = GBOOKS_API_KEY
endpoint = "https://www.googleapis.com/books/v1/volumes"

search_term = "moby dick"
params = {"q": search_term, "maxResults": 3}
response = requests.get(endpoint, params=params).json()
response

{'kind': 'books#volumes',
 'totalItems': 2460,
 'items': [{'kind': 'books#volume',
   'id': 'EIN6AgAAQBAJ',
   'etag': 'fu0v630/GYo',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/EIN6AgAAQBAJ',
   'volumeInfo': {'title': 'Moby Dick',
    'authors': ['Herman Melville'],
    'publisher': 'ABDO Publishing Company',
    'publishedDate': '2010-01-01',
    'description': "In Herman Melville's classic tale of revenge, Ishmael tells his story of becoming a whaler on the Pequod. When Ishmael and his unexpected friend Queequeg join Captain Ahab's hunt for Moby Dick, the voyage of a lifetime turns into tragedy. The adventures of sailing the seas on the hunt for the great white whale is retold in the Calico Illustrated Classics adaptation of Melville's Moby Dick. Calico Chapter Books is an imprint of Magic Wagon, a division of ABDO Group. Grades 3-8.",
    'industryIdentifiers': [{'type': 'ISBN_13', 'identifier': '9781617864124'},
     {'type': 'ISBN_10', 'identifier': '1617864129'}

In [4]:
for book in response["items"]:
    volume = book["volumeInfo"]
    title = volume['title']
    published = volume['publishedDate']
    description = volume['description']
    print(f"{title} ({published})\n{description}\n")

Moby Dick (2010-01-01)
In Herman Melville's classic tale of revenge, Ishmael tells his story of becoming a whaler on the Pequod. When Ishmael and his unexpected friend Queequeg join Captain Ahab's hunt for Moby Dick, the voyage of a lifetime turns into tragedy. The adventures of sailing the seas on the hunt for the great white whale is retold in the Calico Illustrated Classics adaptation of Melville's Moby Dick. Calico Chapter Books is an imprint of Magic Wagon, a division of ABDO Group. Grades 3-8.

Why Read Moby-Dick? (2013-09-24)
A “brilliant and provocative” (The New Yorker) celebration of Melville’s masterpiece—from the bestselling author of In the Heart of the Sea, Valiant Ambition, and In the Hurricane's Eye One of the greatest American novels finds its perfect contemporary champion in Why Read Moby-Dick?, Nathaniel Philbrick’s enlightening and entertaining tour through Melville’s classic. As he did in his National Book Award–winning bestseller In the Heart of the Sea, Philbrick

In [8]:
import os
import configparser
import requests

# Read credentials.cfg file
config = configparser.ConfigParser()
config.read('credentials.cfg')
GBOOKS_API_KEY = config['API']['GOOGLE_BOOKS']

# REPLACE the following variable with your API Key
API_KEY = GBOOKS_API_KEY
endpoint = "https://www.googleapis.com/books/v1/volumes"

search_term = "The Millionaire Next Door"
params = {"key": GBOOKS_API_KEY, "q": search_term, "maxResults": 3}
response = requests.get(endpoint, params=params).json()
response

{'kind': 'books#volumes',
 'totalItems': 2752,
 'items': [{'kind': 'books#volume',
   'id': 'DzytDwAAQBAJ',
   'etag': 'EDqpXQZd23c',
   'selfLink': 'https://www.googleapis.com/books/v1/volumes/DzytDwAAQBAJ',
   'volumeInfo': {'title': 'The Millionaire Next Door',
    'authors': ['Thomas J. Stanley', 'William D. Danko'],
    'publisher': 'Rosetta Books',
    'publishedDate': '2010-11-30',
    'description': 'How do the rich get rich? An updated edition of the “remarkable” New York Times bestseller, based on two decades of research (The Washington Post). Most of the truly wealthy in the United States don’t live in Beverly Hills or on Park Avenue. They live next door. America’s wealthy seldom get that way through an inheritance or an advanced degree. They bargain-shop for used cars, raise children who don’t realize how rich their families are, and reject a lifestyle of flashy exhibitionism and competitive spending. In fact, the glamorous people many of us think of as “rich” are actually 